# LSTM

In [1]:
%load_ext autoreload
%autoreload 2
# В юпитер ноутбуке если вносишь изменения в .py откуда делаются импорты функций и классов, ноутбук прогоняет старые версии, не замечая изменений.
# Эти импорты это исправляют

In [9]:
import pickle                  # Для загрузки словаря
import numpy as np             # математические операции

import torch                                            # фреймворк для машинного обучения и нейросетей
import torch.nn as nn                                   # TensorDataset для создания датасета из Тензора

from aux.rnn_preprocessing import (                         # Импортируем из собственного файла функции для препросессинга
                                data_preprocessing,         # А также модель
                                preprocess_single_string,
                                predict_sentence,
                                LSTMClassifier
                                )

from gensim.models import KeyedVectors # Чтобы прочитать сохраненные векторы

Загружаю вектора сохраненные с коллаб, также загружаю словарь vocab_to_int 

In [10]:
wv = KeyedVectors.load("aux/wv.wordvectors", mmap='r')

with open('aux/vocab_to_int.txt', 'rb') as f:
    vocab_to_int = pickle.load(f)

Создаю embedding layer

In [11]:
# Создаем слой эмбеддинга
# -  wv[word]  обращается к вектору слова  word  в модели Word2Vec
# и присваивает его переменной  embedding_vector .

VOCAB_SIZE = len(vocab_to_int) + 1
EMBEDDING_DIM = 32

embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM))

for word, i in vocab_to_int.items():
    try:
        embedding_vector = wv[word]
        embedding_matrix[i] = embedding_vector
    except KeyError as e:
        # print(f'word: {word}')
        pass


embedding_layer = torch.nn.Embedding.from_pretrained(torch.FloatTensor(embedding_matrix)) # from_pretrained потому что мы его сами тренировали
print(f'EMBEDDING MATRIX SHAPE = number of words x EMEDDING_DIM: {embedding_matrix.shape}')

EMBEDDING MATRIX SHAPE = number of words x EMEDDING_DIM: (196906, 32)


In [12]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cpu'

Импортирую из отдельного файла LSTM и веса, map_location='cpu', т.к. по иначе у меня выдавало ошибку

In [13]:

# EMBEDDING_DIM уже указал при создании embedding_matrix
HIDDEN_DIM = 32
SEQ_LEN = 200
model = LSTMClassifier(embedding_dim=EMBEDDING_DIM, hidden_size=HIDDEN_DIM, embedding=embedding_layer).to(DEVICE)
model.load_state_dict(torch.load('aux/LTSM_model_epoch_7.pt', map_location='cpu'))

<All keys matched successfully>

Предсказываем

In [14]:
SEQ_LEN = 200

pred = predict_sentence('Красивая, чистая больница, доктора добрые, в восторге', model, SEQ_LEN, vocab_to_int)
pred

'positive'

In [15]:
pred = predict_sentence('Некрасивая, грязная больница, доктора злые, в ужасе и печали', model, SEQ_LEN, vocab_to_int)
pred

'negative'